In [3]:
import os
import pandas as pd
import numpy as np
import json
import seaborn as sns
import funcTest as ft
import funcData as fd

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
path_rawdata = 'F://TFG//datasets/raw_datasets//'
path_train = 'F://TFG//datasets//data_train//'
path = 'F:/TFG/datasets/nature-dataset/'


rawdata = pd.read_csv(path_rawdata+'rawDB.csv')
footdata = pd.read_csv(path_rawdata+'fdataDB.csv')
partidos_raw = pd.read_json(path_rawdata+'RAW_partidos.json')

### DATOS EN BRUTO DE NATURE

In [5]:
rawdata[:5]

,wyId,competitionId,seasonId,roundId,dateutc,winner,scoreET_home,coachId_home,side_home,teamId_home,...,scoreHT_home,scoreET_away,coachId_away,side_away,teamId_away,score_away,scoreP_away,hasFormation_away,formation_away,scoreHT_away
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,0,251025,home,3162,...,2,0,101547,away,3161,3,0,1,"{'bench': [{'playerId': 3543, 'ownGoals': '0',...",1
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,0,20603,home,3315,...,0,0,210119,away,3158,1,0,1,"{'bench': [{'playerId': 289122, 'ownGoals': '0...",1
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,0,251044,home,3173,...,0,0,210116,away,3172,0,0,1,"{'bench': [{'playerId': 418317, 'ownGoals': '0...",0
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,0,272970,home,3165,...,0,0,83888,away,3219,0,0,1,"{'bench': [{'playerId': 531179, 'ownGoals': '0...",0
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,0,267555,home,3163,...,1,0,20481,away,3166,0,0,1,"{'bench': [{'playerId': 299557, 'ownGoals': '0...",0


### EXPLORACION

In [6]:
match = rawdata.loc[0]
match

wyId                                                           2576335
competitionId                                                      524
seasonId                                                        181248
roundId                                                        4406278
dateutc                                            2018-05-20 18:45:00
winner                                                            3161
scoreET_home                                                         0
coachId_home                                                    251025
side_home                                                         home
teamId_home                                                       3162
score_home                                                           2
scoreP_home                                                          0
hasFormation_home                                                    1
formation_home       {'bench': [{'playerId': 4792, 'ownGoals': '2',...
scoreH

In [7]:
formacion = match['formation_home']

In [8]:
path = 'F:/TFG/datasets/nature-dataset/'

matches={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open(path + '/matches/matches_%s.json' %nation) as json_data:
        matches[nation] = json.load(json_data)
        
# loading the events data
events={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open(path + '/events/events_%s.json' %nation) as json_data:
        events[nation] = json.load(json_data)

In [9]:
# loading the players data
path = 'F:/TFG/datasets/nature-dataset/'

players={}
with open(path + '/players.json',encoding='utf-8') as json_data:
    players = json.load(json_data)
    
# loading the teams data
teams={}
with open(path + '/teams.json',encoding='utf-8') as json_data:
    teams = json.load(json_data)

In [10]:
partidos = []
eventos  = []

paises = ['Italy','England','Germany','France','Spain']

for pais in paises:
    for m in matches[pais]:
        partidos.append(m)
    for e in events[pais]:
        eventos.append(e)
        
partidos = np.array(partidos,dtype=dict)
eventos = np.array(eventos,dtype=dict)
partidos.shape

(1826,)

In [146]:
tags = pd.read_csv(path+'tags2name.csv')
tags.set_index('Tag',inplace=True,drop=True)

In [147]:
# eventosDF = pd.DataFrame(list(eventos),columns=list(eventos[0].keys()))
eventosDF = pd.read_json(path_rawdata+'eventos_raw.json')
eventosDF.set_index('id',inplace=True)
teams = pd.read_csv(path_rawdata+'teams.csv',index_col='wyId',delimiter=';')
players = pd.read_csv(path_rawdata+'players.csv',index_col='wyId',delimiter=';')

In [148]:
eventos_dict = list(set(zip(eventosDF.eventId,eventosDF.eventName)))
eventos_dict = {v:k for k,v in eventos_dict}
eventos_dict

{'Pass': 8,
 'Offside': 6,
 'Save attempt': 9,
 'Foul': 2,
 'Others on the ball': 7,
 'Goalkeeper leaving line': 4,
 'Shot': 10,
 'Free Kick': 3,
 'Duel': 1,
 'Interruption': 5}

In [149]:
passes = eventosDF[eventosDF.eventId == 8]
shots = eventosDF[eventosDF.eventId == eventos_dict['Shot']]

In [150]:
shots

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId
id,,,,,,,,,,,
180424028,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1203}, {'id'...",21077,"[{'y': 41, 'x': 80}, {'y': 0, 'x': 0}]",2575959,Shot,3172,1H,193.924279,100
180424079,10,Shot,"[{'id': 401}, {'id': 201}, {'id': 1210}, {'id'...",8327,"[{'y': 33, 'x': 95}, {'y': 100, 'x': 100}]",2575959,Shot,3158,1H,250.843152,100
180424208,10,Shot,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",20879,"[{'y': 63, 'x': 72}, {'y': 100, 'x': 100}]",2575959,Shot,3158,1H,633.577774,100
180424444,10,Shot,"[{'id': 1901}, {'id': 402}, {'id': 201}, {'id'...",23149,"[{'y': 62, 'x': 91}, {'y': 100, 'x': 100}]",2575959,Shot,3158,1H,1359.469422,100
180424409,10,Shot,"[{'id': 1901}, {'id': 401}, {'id': 1216}, {'id...",295176,"[{'y': 26, 'x': 86}, {'y': 0, 'x': 0}]",2575959,Shot,3172,1H,1412.601525,100
...,...,...,...,...,...,...,...,...,...,...,...
253302272,10,Shot,"[{'id': 1901}, {'id': 402}, {'id': 2101}, {'id...",267134,"[{'y': 66, 'x': 93}, {'y': 0, 'x': 0}]",2565927,Shot,682,2H,1776.027412,100
253302329,10,Shot,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",267134,"[{'y': 32, 'x': 90}, {'y': 0, 'x': 0}]",2565927,Shot,682,2H,1944.188119,100
253302547,10,Shot,"[{'id': 101}, {'id': 401}, {'id': 201}, {'id':...",134174,"[{'y': 25, 'x': 92}, {'y': 0, 'x': 0}]",2565927,Shot,682,2H,2385.837008,100


#### VAMOS A PROBAR CON ESTADISTICAS DE UN EQUIPO

En este caso usaremos el Atalanta como modo de ejemplo

In [151]:
teams.name[3172]

'Atalanta'

In [152]:
shots_atalanta = shots[shots.teamId == 3172] # atalanta = 3172
passes_atalanta = passes[passes.teamId == 3172]

### Vamos a ver por ejemplo los jugadores con más disparos:

In [153]:
shots_atalanta['counter'] = np.ones(len(shots_atalanta))
shots_atalanta.columns

<ipython-input-153-3ca487ab2f02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots_atalanta['counter'] = np.ones(len(shots_atalanta))


Index(['eventId', 'subEventName', 'tags', 'playerId', 'positions', 'matchId',
       'eventName', 'teamId', 'matchPeriod', 'eventSec', 'subEventId',
       'counter'],
      dtype='object')

In [154]:
top5_shooters_atalanta = shots_atalanta.groupby('playerId').sum('counter').sort_values('counter',ascending=0)[:5]
top5_shooters_atalanta

,eventId,matchId,teamId,eventSec,counter
playerId,,,,,
21077,880,226702467,279136,125796.051632,88.0
20820,780,200938018,247416,106492.935640,78.0
20404,680,175180900,215696,95805.345080,68.0
49991,440,113352937,139568,62198.265437,44.0
246175,390,100468819,123708,49496.794272,39.0


In [155]:
for p in top5_shooters_atalanta.itertuples():
    pl = players.loc[p.Index]
    print(pl.firstName + ' ' + pl.lastName + ' -> ' + str(p.counter))

Alejandro Dar\u00edo G\u00f3mez -> 88.0
Josip Ili\u010di\u0107 -> 78.0
Bryan Cristante -> 68.0
Remo Freuler -> 44.0
Andrea Petagna -> 39.0


#### Jugadores con más porcentaje de acierto de disparo (en Atalanta)

In [156]:
list(shots_atalanta[:20].tags)

[[{'id': 402}, {'id': 201}, {'id': 1203}, {'id': 1801}],
 [{'id': 1901}, {'id': 401}, {'id': 1216}, {'id': 1802}],
 [{'id': 401}, {'id': 201}, {'id': 1201}, {'id': 1801}],
 [{'id': 401}, {'id': 201}, {'id': 1214}, {'id': 1802}],
 [{'id': 401}, {'id': 2101}, {'id': 1802}],
 [{'id': 402}, {'id': 1216}, {'id': 1802}],
 [{'id': 403}, {'id': 2101}, {'id': 1802}],
 [{'id': 402}, {'id': 1212}, {'id': 1802}],
 [{'id': 402}, {'id': 201}, {'id': 1201}, {'id': 1801}],
 [{'id': 403}, {'id': 2101}, {'id': 1802}],
 [{'id': 401}, {'id': 201}, {'id': 1206}, {'id': 1801}],
 [{'id': 401}, {'id': 1210}, {'id': 1802}],
 [{'id': 401}, {'id': 201}, {'id': 1205}, {'id': 1801}],
 [{'id': 402}, {'id': 201}, {'id': 1220}, {'id': 1802}],
 [{'id': 403}, {'id': 201}, {'id': 1213}, {'id': 1802}],
 [{'id': 401}, {'id': 2101}, {'id': 1802}],
 [{'id': 401}, {'id': 2101}, {'id': 1802}],
 [{'id': 403}, {'id': 2101}, {'id': 1802}],
 [{'id': 101}, {'id': 403}, {'id': 201}, {'id': 1204}, {'id': 1801}],
 [{'id': 402}, {'id'

In [157]:
acc = [int('1801' in tag) for tag in list(shots_atalanta.tags)]

In [158]:
shots_atalanta['accuracy'] = acc

<ipython-input-158-dd2b615aa5df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots_atalanta['accuracy'] = acc


In [159]:
top5_acc_atalanta = shots_atalanta.groupby('playerId').mean('accuracy').sort_values('accuracy',ascending=0)[:5]
top5_acc_atalanta

,eventId,matchId,teamId,eventSec,counter,accuracy
playerId,,,,,,
625,10.0,2.576160e+06,3172.0,1633.759932,1.0,0.0
20404,10.0,2.576190e+06,3172.0,1408.902134,1.0,0.0
405597,10.0,2.576329e+06,3172.0,2333.193307,1.0,0.0
373768,10.0,2.576047e+06,3172.0,2488.198871,1.0,0.0
346913,10.0,2.576206e+06,3172.0,848.181631,1.0,0.0


In [160]:
for p in top5_acc_atalanta.itertuples():
    pl = players.loc[p.Index]
    print(pl.firstName + ' ' + pl.lastName + ' -> ' + str(p.accuracy))

Marten de Roon -> 0.0
Bryan Cristante -> 0.0
Alessandro Bastoni -> 0.0
Riccardo Orsolini -> 0.0
Gianluca Mancini -> 0.0


#### Acierto disparos total del equipo (Atalanta)

In [161]:
print(f'Acierto total del equipo: {np.array(acc).mean()}')

Acierto total del equipo: 0.0


# DATASET EVENTOS

In [14]:
# matches = pd.read_csv(path_rawdata+'Nature_wStatsDB_v2.csv',index_col='dateutc')
# matches.index = pd.to_datetime(matches.index)
# matches[:4]

matches = pd.read_csv(path_rawdata+'Nature_wStatsDB_v2.csv')
matches.set_index('wyId',inplace=True)
matches[:4]

,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,...,AS,HST,AST,HC,AC,HF,AF,B365H,B365D,B365A
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,3,...,15,4,6,10,4,12,13,2.40,3.5,2.90
2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,1,...,15,3,6,5,9,14,8,4.00,3.8,1.85
2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,0,...,12,4,4,5,11,17,14,3.10,3.6,2.25
2576330,2018-05-20 16:00:00,524,181248,4406278,3165,3165,1,0,3219,0,...,12,5,1,2,4,13,9,1.53,4.5,5.75


In [163]:
matches.dateutc = pd.to_datetime(matches.dateutc)

Vamos a limpiar el dataset de disparos:
- Posiciones: en un array [y0,x0,y1,x1]
- Accuracy: 1 o 0
- Goal: 1 o 0

In [164]:
# posiciones y tags en formato array
shots['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in shots.itertuples()]
shots['tags'] = np.array([[t['id'] for t in s.tags] for s in shots.itertuples()])

<ipython-input-164-3528d02bdeba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in shots.itertuples()]
<ipython-input-164-3528d02bdeba>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  shots['tags'] = np.array([[t['id'] for t in s.tags] for s in shots.itertuples()])
<ipython-input-164-3528d02bdeba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [165]:
posDF = pd.DataFrame(shots["positions"].to_list(), columns=['y0', 'x0','y1','x1'])
posDF.shape

(40461, 4)

In [166]:
shots.reset_index(inplace=True)

In [167]:
posDF

,y0,x0,y1,x1
0,41,80,0,0
1,33,95,100,100
2,63,72,100,100
3,62,91,100,100
4,26,86,0,0
...,...,...,...,...
40456,66,93,0,0
40457,32,90,0,0
40458,25,92,0,0
40459,30,82,100,100


In [168]:
for c in list(posDF.columns):
    shots[c] = posDF[c]

<ipython-input-168-999ec53f3b66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots[c] = posDF[c]


In [169]:
shots = shots.drop('positions',axis=1)

In [170]:
shots['accuracy'] = [int(1801 in t) for t in shots.tags]
shots['goal'] = [int(101 in t) for t in shots.tags]

In [171]:
shots

,id,eventId,subEventName,tags,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,y0,x0,y1,x1,accuracy,goal
0,180424028,10,Shot,"[402, 201, 1203, 1801]",21077,2575959,Shot,3172,1H,193.924279,100,41,80,0,0,1,0
1,180424079,10,Shot,"[401, 201, 1210, 1802]",8327,2575959,Shot,3158,1H,250.843152,100,33,95,100,100,0,0
2,180424208,10,Shot,"[402, 2101, 1802]",20879,2575959,Shot,3158,1H,633.577774,100,63,72,100,100,0,0
3,180424444,10,Shot,"[1901, 402, 201, 1210, 1802]",23149,2575959,Shot,3158,1H,1359.469422,100,62,91,100,100,0,0
4,180424409,10,Shot,"[1901, 401, 1216, 1802]",295176,2575959,Shot,3172,1H,1412.601525,100,26,86,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40456,253302272,10,Shot,"[1901, 402, 2101, 201, 1802]",267134,2565927,Shot,682,2H,1776.027412,100,66,93,0,0,0,0
40457,253302329,10,Shot,"[402, 2101, 1802]",267134,2565927,Shot,682,2H,1944.188119,100,32,90,0,0,0,0
40458,253302547,10,Shot,"[101, 401, 201, 1205, 1801]",134174,2565927,Shot,682,2H,2385.837008,100,25,92,0,0,1,1
40459,253302642,10,Shot,"[402, 201, 1206, 1801]",3321,2565927,Shot,675,2H,2722.835144,100,30,82,100,100,1,0


In [172]:
# shots.to_json(path_rawdata+'shots.json')
shots = pd.read_json(path_rawdata+'shots.json')

#### Funcion que selecciona los partidos del ultimo mes

In [173]:
def n_matches(db,n):
    ...

### 5 jugadores más disparan cada equipo

De cada partido calculamos los 5 jugadores que mas disparan. Hacemos un dataframe con una columna donde salgan esos 5 jugadores.
Más tarde, para cada equipo podemos sacar los 5 jugadores que más aparezcan en cada lista, en cada partido.

Convendria asignar pesos, ya que alomejor un jugador abarca el 40% de los disparos de un equipo, y por lo tanto no tiene la misma importancia que el resto de los 5 jugadores que más disparan.
A lo mejor podriamos para ello sacar la distribucion de disparos de los jugadores, para calcular el peso de cada jugador.

P.e. La calidad (en cuanto a disparos) de un jugador puede depender de cuantos disparos x partido

Podriamos por ejemplo representar en una gráfica donde x= # jugadores importantes (mas disparan) que le faltan al equipo, y donde y = # goles marcados o resultado (v-e-d) p.e.

In [174]:
shots[-3:]

,id,eventId,subEventName,tags,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,y0,x0,y1,x1,accuracy,goal
40458,253302547,10,Shot,"[101, 401, 201, 1205, 1801]",134174,2565927,Shot,682,2H,2385.837008,100,25,92,0,0,1,1
40459,253302642,10,Shot,"[402, 201, 1206, 1801]",3321,2565927,Shot,675,2H,2722.835144,100,30,82,100,100,1,0
40460,253302686,10,Shot,"[401, 201, 1212, 1802]",4498,2565927,Shot,675,2H,2857.346465,100,43,88,100,100,0,0


### Estadisticas por equipo de disparos por partido.

Resumen de toda la temporada:

In [175]:
team_stats = shots.groupby('teamId').agg(
    y0=('y0','mean'),x0=('x0','mean'),y1=('y1','mean'),x1=('x1','mean'),
    shots_count=('accuracy','count'),acc_count=('accuracy','sum'),
    acc_avg=('accuracy','mean'),
    goal_count=('goal','sum'),goal_avg=('goal','mean'),
)
team_stats.sort_values('goal_count',ascending=False)

,y0,x0,y1,x1,shots_count,acc_count,acc_avg,goal_count,goal_avg
teamId,,,,,,,,,
1625,49.253731,85.270315,47.263682,47.263682,603,250,0.414594,94,0.155887
3767,48.668459,86.275986,50.716846,50.716846,558,233,0.417563,90,0.161290
676,48.272908,87.509960,48.207171,48.207171,502,239,0.476096,87,0.173307
2444,48.502804,85.958879,45.046729,45.046729,535,211,0.394393,83,0.155140
675,48.781300,86.244057,42.313788,42.313788,631,260,0.412044,81,0.128368
...,...,...,...,...,...,...,...,...,...
2450,48.277612,84.486567,42.686567,42.686567,335,121,0.361194,23,0.068657
3315,48.651869,84.420561,47.663551,47.663551,428,127,0.296729,22,0.051402
3783,48.861650,83.097087,42.961165,42.961165,412,114,0.276699,20,0.048544


In [176]:
# hay algunos casos concretos que solo un equipo realiza disparos durante un partido
np.unique(shots.set_index('matchId').loc[2576281].teamId)

array([3161], dtype=int64)

In [177]:
shots_match = shots.groupby(['matchId','teamId']).agg(
    y0=('y0','mean'),x0=('x0','mean'),
    acc_avg=('accuracy','mean'),
    goal_avg=('goal','mean'),
)

shots_match

y0         x0   acc_avg  goal_avg
matchId teamId                                          
2499719 1609    48.555556  82.074074  0.370370  0.148148
        1631    54.142857  89.428571  0.571429  0.428571
2499720 1625    45.416667  83.583333  0.250000  0.083333
        1651    48.333333  82.666667  0.333333  0.000000
2499721 1610    42.066667  81.000000  0.333333  0.133333
...                   ...        ...       ...       ...
2576336 3315    46.071429  82.928571  0.214286  0.000000
2576337 3164    47.000000  83.333333  0.333333  0.111111
        3204    51.071429  84.000000  0.357143  0.142857
2576338 3185    58.250000  92.250000  0.500000  0.500000
        3193    49.363636  86.727273  0.181818  0.090909

[3649 rows x 4 columns]

In [178]:
sm_aux1 = shots_match.reset_index()[::2]
sm_aux2 = shots_match.reset_index()[1::2]

In [179]:
shots_match = pd.merge(sm_aux1,sm_aux2,on='matchId',how='outer',suffixes=('_sh_H','_sh_A')).set_index('matchId')
shots_match


,teamId_sh_H,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,teamId_sh_A,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A
matchId,,,,,,,,,,
2499719,1609.0,48.555556,82.074074,0.370370,0.148148,1631.0,54.142857,89.428571,0.571429,0.428571
2499720,1625.0,45.416667,83.583333,0.250000,0.083333,1651.0,48.333333,82.666667,0.333333,0.000000
2499721,1610.0,42.066667,81.000000,0.333333,0.133333,1646.0,41.222222,91.111111,0.555556,0.333333
2499722,1628.0,44.000000,89.363636,0.272727,0.000000,1673.0,48.125000,85.000000,0.750000,0.250000
2499723,1623.0,50.750000,86.500000,0.375000,0.125000,1639.0,51.800000,83.000000,0.100000,0.000000
...,...,...,...,...,...,...,...,...,...,...
2576335,3162.0,53.071429,86.500000,0.285714,0.071429,3161.0,48.571429,84.857143,0.428571,0.142857
2576336,3315.0,46.071429,82.928571,0.214286,0.000000,3158.0,45.133333,84.600000,0.400000,0.000000
2576337,3204.0,51.071429,84.000000,0.357143,0.142857,3164.0,47.000000,83.333333,0.333333,0.111111


In [180]:
shots_match[shots_match.isna().any(axis=1)].reset_index()

,matchId,teamId_sh_H,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,teamId_sh_A,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A
0,2499815,1612.0,47.500000,87.571429,0.500000,0.214286,NaN,NaN,NaN,NaN,NaN
1,2576281,3161.0,42.157895,87.736842,0.421053,0.157895,NaN,NaN,NaN,NaN,NaN
2,2500013,NaN,NaN,NaN,NaN,NaN,1673.0,51.074074,85.851852,0.111111,0.0


In [181]:
matches['matchId'] = matches.index.to_list()
# matches.drop('wyId',axis=1,inplace=True)

In [182]:
for row in shots_match[shots_match.isna().any(axis=1)].reset_index().itertuples():
    shots_match.loc[row.matchId].teamId_sh1 = matches.loc[row.matchId].teamId_home
    shots_match.loc[row.matchId].teamId_sh2 = matches.loc[row.matchId].teamId_away

In [183]:
shots_match[shots_match.isna().any(axis=1)].reset_index()

,matchId,teamId_sh_H,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,teamId_sh_A,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A
0,2499815,1612.0,47.500000,87.571429,0.500000,0.214286,NaN,NaN,NaN,NaN,NaN
1,2576281,3161.0,42.157895,87.736842,0.421053,0.157895,NaN,NaN,NaN,NaN,NaN
2,2500013,NaN,NaN,NaN,NaN,NaN,1673.0,51.074074,85.851852,0.111111,0.0


En vez de eliminar las filas con valores NaN vamos a sustituir los valores con el ID del equipo, pero los valores de y0 x0 y de accuracy lo dejamos a NaN
Cuando se utilice en un modelo como input habrá que filtrar dichos valores! Sin embargo, para crear los datos de los partidos posteriores si que es útil tenerlo.

In [184]:
matches_stats = pd.merge(matches.reset_index(),shots_match.reset_index(),on='matchId',how='inner')
matches_stats.set_index('matchId',inplace=True, drop=True)
matches_stats.drop(['teamId_sh_H', 'teamId_sh_A'],axis=1,inplace=True)

In [185]:
matches_stats[matches_stats.isna().any(axis=1)]

,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,B365D,B365A,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A
matchId,,,,,,,,,,,,,,,,,,,,,
2576281,2576281,2018-04-17 18:45:00,524,181248,4406278,3161,3161,4,1,3173,...,7.00,15.0,42.157895,87.736842,0.421053,0.157895,NaN,NaN,NaN,NaN
2500013,2500013,2018-03-10 15:00:00,364,181150,4405654,0,1673,0,0,10531,...,3.10,3.2,NaN,NaN,NaN,NaN,51.074074,85.851852,0.111111,0.0
2499815,2499815,2017-10-28 14:00:00,364,181150,4405654,1612,1612,3,0,1673,...,6.75,13.0,47.500000,87.571429,0.500000,0.214286,NaN,NaN,NaN,NaN


In [186]:
# matches_stats.to_csv(path_rawdata+'match_stats.csv',sep=';')
matches_stats = pd.read_csv(path_rawdata+'match_stats.csv',sep=';',index_col='matchId')
# matches_stats.drop(['teamId_sh1', 'teamId_sh2'],axis=1,inplace=True)
matches_stats[:3]

,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A,mins4_H,mins4_A
matchId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,53.071429,86.500000,0.285714,0.071429,48.571429,84.857143,0.428571,0.142857,65.000000,67.386364
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,46.071429,82.928571,0.214286,0.000000,45.133333,84.600000,0.400000,0.000000,60.181818,44.568182
2576329,2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,...,47.833333,83.000000,0.666667,0.166667,48.181818,86.818182,0.363636,0.000000,63.113636,55.204545


In [187]:
matches_stats.columns

Index(['wyId', 'dateutc', 'competitionId', 'seasonId', 'roundId', 'winner',
       'teamId_home', 'score_home', 'scoreHT_home', 'teamId_away',
       'score_away', 'scoreHT_away', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'B365H', 'B365D', 'B365A', 'y0_sh_H', 'x0_sh_H',
       'acc_avg_sh_H', 'goal_avg_sh_H', 'y0_sh_A', 'x0_sh_A', 'acc_avg_sh_A',
       'goal_avg_sh_A', 'mins4_H', 'mins4_A'],
      dtype='object')

#### Sacamos los jugadores con mas disparos en un partido

In [188]:
n = 4
mtc = 2576143
tid = matches.loc[mtc].teamId_home
mask_time = (matches.loc[mtc].dateutc > matches.dateutc)
mask_team_h = (matches.teamId_home == tid)
mask = mask_time & mask_team_h
# mask_team_a = (matches.teamId_away == tid)

# TODO: hacer loop que recorra cada partido, y hacer el proceso para los dos equipos de cada partido

In [189]:
partidos_anteriores = matches[mask][:n].index.to_list()

In [190]:
players_shots = shots[shots.matchId == partidos_anteriores[0]].playerId.to_numpy()
np.argsort(np.bincount(players_shots))[::-1][:4]

array([ 21842, 269152, 447804,  25541], dtype=int64)

In [12]:
def ult_partidos(npj,nj=4):

    partidos_h = {}
    partidos_a = {}

    for mtc in matches.index.to_list():
        tid_h, tid_a = matches.loc[mtc].teamId_home, matches.loc[mtc].teamId_away
        mask_time = (matches.loc[mtc].dateutc > matches.dateutc)
        mask_team_h, mask_team_a = (matches.teamId_home == tid_h), (matches.teamId_away == tid_a)
        mask_h, mask_a = mask_time & mask_team_h, mask_time & mask_team_a
        # hecho para que solo coja los ultimos partidos en casa si el partido actual es en casa. Igual para visitante.


        if sum(mask_h) >= npj:
            partidos_anteriores_h = (matches[mask_h][:npj].index).to_numpy()
            partidos_anteriores_a = (matches[mask_a][:npj].index).to_numpy()
            # # 
            # # calcular top shooters:
            # matches_stats.loc[mtc].top_shooters_h = np.argsort(np.bincount(shots[shots.matchId.isin(partidos_anteriores_h)].playerId))[::-1][:nj]
            # matches_stats.loc[mtc].top_shooters_a = np.argsort(np.bincount(shots[shots.matchId.isin(partidos_anteriores_a)].playerId))[::-1][:nj]
        else:
            partidos_anteriores_h, partidos_anteriores_a = np.nan, np.nan

        partidos_h[mtc], partidos_a[mtc] = partidos_anteriores_h, partidos_anteriores_a

        # jugadores mas disparos

    return partidos_h, partidos_a

In [192]:
matches[matches.teamId_home == 3162][:10]

,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,...,HST,AST,HC,AC,HF,AF,B365H,B365D,B365A,matchId
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,3,...,4,6,10,4,12,13,2.40,3.50,2.9,2576335
2576309,2018-05-06 13:00:00,524,181248,4406278,0,3162,1,1,3172,1,...,2,6,6,9,7,8,2.04,3.60,3.5,2576309
2576297,2018-04-22 13:00:00,524,181248,4406278,3162,3162,4,2,3164,0,...,9,2,8,4,11,8,1.30,6.00,9.0,2576297
2576274,2018-04-15 18:45:00,524,181248,4406278,0,3162,0,0,3158,0,...,1,1,7,2,14,13,2.79,3.39,2.5,2576274
2576249,2018-03-31 13:00:00,524,181248,4406278,3162,3162,6,1,3219,2,...,14,3,8,2,14,11,1.16,7.00,15.0,2576249
2576240,2018-03-18 19:45:00,524,181248,4406278,0,3162,1,1,3166,1,...,4,3,5,3,7,9,1.33,4.75,9.5,2576240
2576225,2018-03-03 17:00:00,524,181248,4406278,3159,3162,0,0,3159,1,...,2,1,4,4,16,11,3.50,3.29,2.2,2576225
2576202,2018-02-19 19:45:00,524,181248,4406278,3162,3162,2,0,3194,0,...,10,0,11,2,7,10,1.16,7.50,17.0,2576202
2576182,2018-02-05 19:45:00,524,181248,4406278,3193,3162,1,0,3193,2,...,5,4,8,1,7,9,1.50,4.20,7.0,2576182


In [193]:
partidos_h, partidos_a = ult_partidos(5,4)

In [194]:
np.argsort(np.bincount(shots[shots.matchId.isin(partidos_h[2576335])].playerId))[::-1][:4]

array([21384,  7965, 40806, 21350], dtype=int64)

In [195]:
matches_stats.shape

(1826, 33)

In [196]:
matches.shape

(1826, 23)

### Repetimos lo mismo pero con los pases

In [197]:
passes[:3]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId
id,,,,,,,,,,,
180423957,8,Simple pass,[{'id': 1801}],8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959,Pass,3158,1H,2.530536,85
180423958,8,Simple pass,[{'id': 1801}],20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959,Pass,3158,1H,3.768418,85
180423962,8,Simple pass,[{'id': 1801}],8306,"[{'y': 15, 'x': 72}, {'y': 25, 'x': 72}]",2575959,Pass,3158,1H,10.376066,85


In [198]:
# posiciones y tags en formato array
passes['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in passes.itertuples()]
passes['tags'] = [[t['id'] for t in s.tags] for s in passes.itertuples()]

<ipython-input-198-21ad0b92b567>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in passes.itertuples()]
<ipython-input-198-21ad0b92b567>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes['tags'] = [[t['id'] for t in s.tags] for s in passes.itertuples()]


In [199]:
posDF = pd.DataFrame(passes["positions"].to_list(), columns=['y0', 'x0','y1','x1'])
print(posDF.shape)
passes.reset_index(inplace=True)
posDF

(1565356, 4)


,y0,x0,y1,x1
0,52,49,44,44
1,44,43,17,17
2,15,72,25,25
3,82,38,88,88
4,15,61,15,15
...,...,...,...,...
1565351,29,8,19,19
1565352,34,36,52,52
1565353,25,66,2,2
1565354,21,97,26,26


In [200]:
for c in list(posDF.columns):
    passes[c] = posDF[c]

<ipython-input-200-d1a22ff0b32a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes[c] = posDF[c]


In [201]:
subEventDictPass = { id:name for id,name in zip(np.unique(passes.subEventId),np.unique(passes.subEventName)) }
subEventDictPass

{80: 'Cross',
 81: 'Hand pass',
 82: 'Head pass',
 83: 'High pass',
 84: 'Launch',
 85: 'Simple pass',
 86: 'Smart pass'}

In [202]:
# passes = passes.drop('positions',axis=1)
passes[passes.subEventId == 86]

,id,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,y0,x0,y1,x1
13,180424000,8,Smart pass,[1802],49991,"[12, 58, 9, 9]",2575959,Pass,3172,1H,101.978336,86,12,58,9,9
128,180424296,8,Smart pass,[1802],3475,"[24, 69, 51, 51]",2575959,Pass,3158,1H,893.852993,86,24,69,51,51
207,180424439,8,Smart pass,"[302, 1901, 1801]",44251,"[77, 69, 62, 62]",2575959,Pass,3158,1H,1357.201318,86,77,69,62,62
292,180424628,8,Smart pass,"[901, 1802]",49991,"[17, 75, 26, 26]",2575959,Pass,3172,1H,2134.626049,86,17,75,26,26
318,180424723,8,Smart pass,[1801],3475,"[30, 80, 30, 30]",2575959,Pass,3158,1H,2277.245050,86,30,80,30,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565183,253302265,8,Smart pass,"[302, 1901, 901, 1801]",3529,"[95, 75, 66, 66]",2565927,Pass,682,2H,1773.065301,86,95,75,66,66
1565203,253302367,8,Smart pass,"[901, 1802]",40756,"[36, 65, 45, 45]",2565927,Pass,675,2H,1917.515642,86,36,65,45,45
1565220,253302420,8,Smart pass,"[901, 1802]",3321,"[67, 76, 59, 59]",2565927,Pass,675,2H,2030.319042,86,67,76,59,59
1565256,253302456,8,Smart pass,"[901, 1802]",134174,"[65, 70, 63, 63]",2565927,Pass,682,2H,2195.640788,86,65,70,63,63


In [203]:
passes['accuracy'] = np.array([(1801 in t) for t in passes.tags])*1
passes['keyPass'] = np.array([(302 in t) for t in passes.tags])*1

<ipython-input-203-8772cf182f7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes['accuracy'] = np.array([(1801 in t) for t in passes.tags])*1
<ipython-input-203-8772cf182f7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes['keyPass'] = np.array([(302 in t) for t in passes.tags])*1


In [204]:
passes

,id,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,y0,x0,y1,x1,accuracy,keyPass
0,180423957,8,Simple pass,[1801],8327,"[52, 49, 44, 44]",2575959,Pass,3158,1H,2.530536,85,52,49,44,44,1,0
1,180423958,8,Simple pass,[1801],20438,"[44, 43, 17, 17]",2575959,Pass,3158,1H,3.768418,85,44,43,17,17,1,0
2,180423962,8,Simple pass,[1801],8306,"[15, 72, 25, 25]",2575959,Pass,3158,1H,10.376066,85,15,72,25,25,1,0
3,180423979,8,Simple pass,[1801],86366,"[82, 38, 88, 88]",2575959,Pass,3172,1H,16.240563,85,82,38,88,88,1,0
4,180423968,8,Simple pass,[1801],8306,"[15, 61, 15, 15]",2575959,Pass,3158,1H,19.153235,85,15,61,15,15,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565351,253302662,8,Hand pass,[1801],3486,"[29, 8, 19, 19]",2565927,Pass,682,2H,2925.547631,81,29,8,19,19,1,0
1565352,253302665,8,Simple pass,[1801],3695,"[34, 36, 52, 52]",2565927,Pass,682,2H,2931.782904,85,34,36,52,52,1,0
1565353,253302671,8,Simple pass,[1801],20623,"[25, 66, 2, 2]",2565927,Pass,682,2H,2939.077491,85,25,66,2,2,1,0
1565354,253302674,8,Cross,"[401, 1802]",122832,"[21, 97, 26, 26]",2565927,Pass,682,2H,2942.098761,80,21,97,26,26,0,0


In [205]:
# passes.to_json(path_rawdata+'passes.json')
passes = pd.read_json(path_rawdata+'passes.json')

In [206]:
passes_match = passes.groupby(['matchId','teamId']).agg(
    y0=('y0','mean'),x0=('x0','mean'),
    y1=('y1','mean'),x1=('x1','mean'),
    acc_avg=('accuracy','mean'),
    keypass=('keyPass','sum'),
)

passes_match

y0         x0         y1         x1   acc_avg  keypass
matchId teamId                                                               
2499719 1609    48.247525  53.176568  47.627063  47.627063  0.846535        5
        1631    47.308696  49.369565  49.295652  49.295652  0.721739        2
2499720 1625    51.200265  55.713528  50.765252  50.765252  0.904509        3
        1651    52.358696  39.445652  51.826087  51.826087  0.673913        1
2499721 1610    56.063953  47.947674  54.399225  54.399225  0.879845        2
...                   ...        ...        ...        ...       ...      ...
2576336 3315    48.296736  47.931751  47.008902  47.008902  0.777448        2
2576337 3164    53.278027  49.408072  54.295964  54.295964  0.865471        0
        3204    52.308540  48.663912  51.479339  51.479339  0.867769        1
2576338 3185    49.097633  39.713018  50.985207  50.985207  0.849112        1
        3193    51.357143  54.415414  51.847744  51.847744  0.866541        4

[3652 rows x 6 columns]

In [207]:
sm_aux1 = passes_match.reset_index()[::2]
sm_aux2 = passes_match.reset_index()[1::2]

In [208]:
passes_match = pd.merge(sm_aux1,sm_aux2,on='matchId',how='outer',suffixes=('_ps_H','_ps_A')).set_index('matchId')
passes_match

,teamId_ps_H,y0_ps_H,x0_ps_H,y1_ps_H,x1_ps_H,acc_avg_ps_H,keypass_ps_H,teamId_ps_A,y0_ps_A,x0_ps_A,y1_ps_A,x1_ps_A,acc_avg_ps_A,keypass_ps_A
matchId,,,,,,,,,,,,,,
2499719,1609,48.247525,53.176568,47.627063,47.627063,0.846535,5,1631,47.308696,49.369565,49.295652,49.295652,0.721739,2
2499720,1625,51.200265,55.713528,50.765252,50.765252,0.904509,3,1651,52.358696,39.445652,51.826087,51.826087,0.673913,1
2499721,1610,56.063953,47.947674,54.399225,54.399225,0.879845,2,1646,46.638514,46.479730,45.391892,45.391892,0.827703,1
2499722,1628,48.481081,45.337838,48.308108,48.308108,0.786486,1,1673,49.553114,47.849817,48.333333,48.333333,0.710623,0
2499723,1623,54.437759,47.126556,54.298755,54.298755,0.842324,3,1639,49.475655,52.198502,49.322097,49.322097,0.767790,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576334,3159,46.656846,51.795494,46.792028,46.792028,0.896014,4,3194,47.680702,45.263158,49.915789,49.915789,0.835088,0
2576335,3161,47.347561,47.630081,46.804878,46.804878,0.847561,3,3162,44.052980,46.741722,43.966887,43.966887,0.824503,4
2576336,3158,47.292000,45.682000,47.494000,47.494000,0.846000,2,3315,48.296736,47.931751,47.008902,47.008902,0.777448,2


In [209]:
passes_match[passes_match.isna().any(axis=1)].reset_index()

# no hay ningun valor indefinido NaN

,matchId,teamId_ps_H,y0_ps_H,x0_ps_H,y1_ps_H,x1_ps_H,acc_avg_ps_H,keypass_ps_H,teamId_ps_A,y0_ps_A,x0_ps_A,y1_ps_A,x1_ps_A,acc_avg_ps_A,keypass_ps_A


In [210]:
matches_stats = pd.merge(matches_stats,passes_match.reset_index(),on='matchId',how='inner')
# matches_stats.set_index('matchId',inplace=True, drop=True)
matches_stats.drop(['teamId_ps_H', 'teamId_ps_A'],axis=1,inplace=True)

In [211]:
matches_stats.columns

Index(['matchId', 'wyId', 'dateutc', 'competitionId', 'seasonId', 'roundId',
       'winner', 'teamId_home', 'score_home', 'scoreHT_home', 'teamId_away',
       'score_away', 'scoreHT_away', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'B365H', 'B365D', 'B365A', 'y0_sh_H', 'x0_sh_H',
       'acc_avg_sh_H', 'goal_avg_sh_H', 'y0_sh_A', 'x0_sh_A', 'acc_avg_sh_A',
       'goal_avg_sh_A', 'mins4_H', 'mins4_A', 'y0_ps_H', 'x0_ps_H', 'y1_ps_H',
       'x1_ps_H', 'acc_avg_ps_H', 'keypass_ps_H', 'y0_ps_A', 'x0_ps_A',
       'y1_ps_A', 'x1_ps_A', 'acc_avg_ps_A', 'keypass_ps_A'],
      dtype='object')

## Total minutos acumulados de la alineacion

Sumamos los minutos jugados en los ultimos N partidos por los jugadores que componen la alineacion inicial del partido

### Crear tabla de las Formaciones

In [212]:
partidos_raw[:3].teamsData

wyId
2576335    {'3162': {'scoreET': 0, 'coachId': 251025, 'si...
2576336    {'3158': {'scoreET': 0, 'coachId': 210119, 'si...
2576329    {'3173': {'scoreET': 0, 'coachId': 251044, 'si...
Name: teamsData, dtype: object

Vamos a hacer un DataFrame que indique los IDs de los jugadores que juegan y otro DataFrame con los minutos.
Tendremos dos DF que cada posicion concuerda con el jugador y tiempo de juego

In [215]:
# partidos_raw.set_index('wyId',inplace=True)

In [216]:
print(partidos_raw.columns)
assert matches.reset_index().wyId.all() == partidos_raw.reset_index().wyId.all()

Index(['status', 'roundId', 'gameweek', 'teamsData', 'seasonId', 'dateutc',
       'winner', 'venue', 'label', 'date', 'referees', 'duration',
       'competitionId'],
      dtype='object')


Creacion de las dos tablas

In [217]:
x = {'a': 1, 'b': 2}
y = {'d': 10, 'c': 11}

z = {**x, **y}
list(z.values())

[1, 2, 10, 11]

In [218]:
playerIds = np.array(players.index)

In [219]:
players_match_mins = {pId:{} for pId in playerIds}    # creamos un dict vacio para cada jugador

In [220]:
matches

,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,...,HST,AST,HC,AC,HF,AF,B365H,B365D,B365A,matchId
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,3,...,4,6,10,4,12,13,2.40,3.50,2.90,2576335
2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,1,...,3,6,5,9,14,8,4.00,3.80,1.85,2576336
2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,0,...,4,4,5,11,17,14,3.10,3.60,2.25,2576329
2576330,2018-05-20 16:00:00,524,181248,4406278,3165,3165,1,0,3219,0,...,5,1,2,4,13,9,1.53,4.50,5.75,2576330
2576331,2018-05-20 16:00:00,524,181248,4406278,3163,3163,1,1,3166,0,...,5,5,10,2,9,11,1.53,4.33,6.00,2576331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565552,2017-08-19 20:15:00,795,181144,4406122,0,680,1,1,691,1,...,4,6,7,3,14,12,1.62,4.00,5.50,2565552
2565551,2017-08-19 18:15:00,795,181144,4406122,0,756,2,2,679,2,...,6,3,6,0,15,15,8.00,4.33,1.45,2565551
2565549,2017-08-19 16:15:00,795,181144,4406122,687,692,2,1,687,3,...,5,6,5,4,12,11,2.38,3.25,3.20,2565549


In [221]:
indice          = [] # idmatch_idteam
players_id_mins = []
players_mins    = []

# tambien estamos interesados en mantener un diccionario con todos los minutos que han jugado cada jugador y en que partidos
players_match_mins = {pId:{} for pId in playerIds}    # creamos un dict vacio para cada jugador

for m in matches.reset_index().wyId:
    form_info = partidos_raw.teamsData.loc[m]                                   # seleccionamos el campo de las formaciones
    for t in list(form_info.keys()):                                            # iteramos sobre los dos equipos
        indice.append((m,int(t)))                                               # guardamos en una lista de tuplas lo que seran las keys de la tabla (partido,equipo)
        form_t = form_info[t]['formation']                                      # seleccionamos la formacion de uno de los equipos
        dict_team1 = {pl['playerId']:0 for pl in form_t['bench']}              
        dict_team2 = {pl['playerId']:90 for pl in form_t['lineup']}
        dict_match = {**dict_team1, **dict_team2}
        # añadimos los minutos en el dict de los minutos de cada jugador a lo largo de la temporada
        for pl in form_t['bench']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 0
        for pl in form_t['lineup']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 90
        # print((m,int(t)))
        if form_t['substitutions'] != 'null':
            for subs in form_t['substitutions']:
                # print(subs['minute'])
                if players_match_mins.get(subs['playerIn'],-1) != -1 and players_match_mins.get(subs['playerOut'],-1) != -1:
                    mins = subs['minute'] if (subs['minute'] < 90) else 90
                    dict_match[subs['playerIn']] = 90-mins
                    players_match_mins[subs['playerIn']][m] = 90-mins  
                    dict_match[subs['playerOut']] = dict_match[subs['playerOut']] - (90-mins)  
                    players_match_mins[subs['playerOut']][m] = dict_match[subs['playerOut']] - (90-mins) 
                

        players_id_mins.append(np.array(list(dict_match.keys())))   
        players_mins.append(np.array(list(dict_match.values())))   

players_id_mins = np.array(players_id_mins)
players_mins = np.array(players_mins)


<ipython-input-221-d195c76ace06>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  players_id_mins = np.array(players_id_mins)
<ipython-input-221-d195c76ace06>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  players_mins = np.array(players_mins)


In [222]:
players_match_mins[372752][2565608]

90

In [223]:
# comprobamos que todos las 3 listas tengan el mismo tamaño
assert len(players_id_mins) == len(players_mins) == len(indice)

In [224]:
# todos los equipos deberian sumar 990 minutos en un partido entre sus jugadores

for mId,mins in zip(indice,players_mins):
    if np.sum(mins) != 990:
        print(mId, mins)
    # assert np.sum(m) == 11*90

# PARTIDOS QUE NO CUMPLEN LOS MINUTOS:
# (2575965, 3204) [ 0  0  0  0  0  0  0  0  0  0  0  0 56 62 88 90 90 90 90 90 90 90 90  2] -> no estan registrados los IDs de los jugadores que entran al partido
# (2575959, 3158) [ 0  0  0  0  0  0  0  0  0  0  0 74 81 84 90 90 90 90 90 90 90 90  6] -> no estan registrados los IDs de los jugadores que entran al partido

In [225]:
# partidos_raw.loc[2575959].teamsData

In [226]:
indice_list = np.array(indice)
indice_list

array([[2576335,    3162],
       [2576335,    3161],
       [2576336,    3158],
       ...,
       [2565556,     714],
       [2565557,     696],
       [2565557,     712]])

In [227]:
# vamos a crear un dataframe con dichos arrays

players_mins_DF = pd.DataFrame({'matchesId': indice_list[:,0], 'teamsId': indice_list[:,1], 'players_id': players_id_mins, 'players_min': players_mins})
players_mins_DF.set_index(['matchesId','teamsId'],drop=True,inplace=True)
players_mins_DF

players_id  \
matchesId teamsId                                                      
2576335   3162     [4792, 20972, 256479, 241675, 260250, 69417, 2...   
          3161     [3543, 405608, 20572, 86785, 7982, 70161, 2982...   
2576336   3158     [289122, 20438, 20518, 137298, 350032, 397239,...   
          3315     [423151, 267185, 22699, 392384, 302798, 415348...   
2576329   3173     [402898, 434142, 21299, 20636, 40538, 283832, ...   
...                                                              ...   
2565549   687      [3663, 3667, 225996, 477352, 3685, 3576, 3681,...   
2565556   674      [326535, 215265, 3843, 364844, 293862, 250885,...   
          714      [3372, 4350, 4362, 519496, 4361, 4238, 86200, ...   
2565557   696      [279541, 286981, 224080, 91064, 114055, 3650, ...   
          712      [6513, 484931, 86716, 86350, 6365, 167118, 393...   

                                                         players_min  
matchesId teamsId                                                     
2576335   3162     [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 13, 15, 75, 77,...  
          3161     [0, 0, 0, 0, 0, 0, 0, 0, 23, 30, 11, 60, 67, 7...  
2576336   3158     [0, 0, 0, 0, 0, 0, 0, 0, 13, 16, 21, 69, 74, 7...  
          3315     [0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 8, 20, 59, 70,...  
2576329   3173     [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 19, 15, 71, 75,...  
...                                                              ...  
2565549   687      [0, 0, 0, 0, 33, 23, 8, 57, 67, 82, 90, 90, 90...  
2565556   674      [0, 0, 0, 0, 17, 11, 1, 73, 79, 89, 90, 90, 90...  
          714      [0, 0, 0, 0, 12, 5, 12, 78, 78, 85, 90, 90, 90...  
2565557   696      [0, 0, 0, 0, 28, 27, 15, 62, 63, 75, 90, 90, 9...  
          712      [0, 0, 0, 0, 10, 25, 20, 65, 70, 80, 90, 90, 9...  

[3652 rows x 2 columns]

In [228]:
players_mins_DF.loc[(2576281, 3173)]

players_id     [475356, 472938, 21299, 22933, 21639, 488648, ...
players_min    [0, 0, 0, 0, 0, 0, 0, 0, 18, 38, 41, 49, 52, 7...
Name: (2576281, 3173), dtype: object

In [229]:
partidos_raw.loc[2576281]

status                                                      Played
roundId                                                    4406278
gameweek                                                        33
teamsData        {'3173': {'scoreET': 0, 'coachId': 251044, 'si...
seasonId                                                    181248
dateutc                                        2018-04-17 18:45:00
winner                                                        3161
venue                                                             
label                             Internazionale - Cagliari, 4 - 0
date                            April 17, 2018 at 8:45:00 PM GMT+2
referees         [{'refereeId': 377246, 'role': 'referee'}, {'r...
duration                                                   Regular
competitionId                                                  524
Name: 2576281, dtype: object

In [230]:
print(players_match_mins[393228].values())
print(len(players_match_mins[393228].keys()))


dict_values([0, 0, 0, 44, 4, 4, 0, 6, 90, 6, 0, 90, 90, 90, 90, 90, 90, 30, 90, 0, 90, 90, 0, 44, 0, 62, 0, 0, 44, 0, 50, 90])
32


## Vector de caracteristicas para cada equipo, solo teniendo en cuenta los jugadores de la alineacion

- Features defensivas -> tackles, fouls, interception, clearances...
- Features portero -> saves attempts
- Features ataque -> smart passes, cross passes, duels
- Features generales -> missed balls, time loss,  

Vamos a crear estadisticas de cada equipo en cada partido, partiendo de los jugadores del lineup.

Evaluaremos:
- Estadisticas como valor absoluto -> i.e. # smart passes
- Estadisticas por minutos jugados (sin weighted) -> i.e. # smart passes / # minutos jugados
- Estadisticas por minutos jugados (weighted) -> i.e. # smart passes * # minutos jugados / # minutos jugados

In [231]:
def _2means(m,x):
    print((1/m @ x) / np.sum(1/m), np.mean(x))
    print()


#### VECTOR MINUTOS X EQUIPO

Vamos a crear dos vectores: uno con el numero de minutos acumulados en los últimos N partidos por los jugadores titulares del partido, el otro vector será lo mismo pero dividiendo por el numero total de minutos disputados en los ultimos N partidos (N * 90 mins).

También puede ser interesante crear un vector con unos pesos que multipliquen los minutos disputados. Los pesos representarian los resultados de los últimos N partidos jugados por el jugador.

In [232]:
# vamos a añadir a matches_stats dos vectores (uno para cada equipo) con el % de minutos disputados de cada equipo

matches_stats.head(3)

,matchId,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,...,y1_ps_H,x1_ps_H,acc_avg_ps_H,keypass_ps_H,y0_ps_A,x0_ps_A,y1_ps_A,x1_ps_A,acc_avg_ps_A,keypass_ps_A
0,2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,...,46.804878,46.804878,0.847561,3,44.052980,46.741722,43.966887,43.966887,0.824503,4
1,2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,...,47.494000,47.494000,0.846000,2,48.296736,47.931751,47.008902,47.008902,0.777448,2
2,2576329,2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,...,47.553663,47.553663,0.873935,4,53.612069,47.198276,50.568966,50.568966,0.745690,2


In [233]:
# vamos a calcular los vectores a partir de los últimos npj partidos
npj = 4
partidos_h, partidos_a = ult_partidos(npj=npj)

In [234]:
players_mins_DF.loc[2576335,3162].players_id[:-11]

array([  4792,  20972, 256479, 241675, 260250,  69417,  20460, 101635,
        20575,   7926, 208865,  37745])

In [235]:
partidos_h[2565632]

array([2565608, 2565595, 2565570, 2565556], dtype=int64)

In [236]:
# seleccionamos solo los partidos que tienen 4 partidos anteriores
idxs = np.array(pd.DataFrame(partidos_h).T.reset_index().dropna(axis=0)['index'].to_list())
matches_wUltPartidos = matches_stats.set_index('wyId').loc[idxs]

In [237]:
# iteramos sobre cada partido de matches_stats y para cada equipo seleccionamos los jugadores titulares.

minsH, minsA = np.zeros(len(matches_wUltPartidos)), np.zeros(len(matches_wUltPartidos))

for i,m in enumerate(matches_wUltPartidos.reset_index().itertuples()):
    # if m.wyId == 2565632: print(m.wyId)
    # los jugadores titulares son los últimos 11 elementos de los arrays 'players_id'
    lineup_h = players_mins_DF.loc[m.wyId, m.teamId_home].players_id[-11:]
    lineup_a = players_mins_DF.loc[m.wyId, m.teamId_away].players_id[-11:]

    for player in lineup_h:
        for part in partidos_h[m.wyId]:
            if part == 2565632: print(player)
            minsH[i] += players_match_mins[player].get(part,0)

    for player in lineup_a:
        for part in partidos_a[m.wyId]:
            minsA[i] += players_match_mins[player].get(part,0)

    # minsH = minsH / (npj*90)
    # minsA = minsA / (npj*90)

    # OPCION CODIGO
    # crear un diccionario grande de los ultimos N partidos (playerId : player_min) y hacer ReduceByKey(add), con lo cual tendremos la suma de minutos de cada jugador
    # a continuación escogeriamos solamente las Keys de los jugadores del lineup y las sumariamos
        

326535
237034
3339
22202
3286
20922
70112
240445
41363
25623
70129
372752
70129
4892
3339
22202
3286
20922
211853
240445
41363
25623
70129
70112
372752
237034
4892
3339
22202
3286
20922
41363
25623
372752
22202
70129
237034
4892
3339
3286
20922
70112
4065
25623


In [238]:
assert len(minsA) == len(minsH) == len(matches_wUltPartidos)

In [239]:
matches_wUltPartidos['mins4_H'] = minsH / (npj*11)
matches_wUltPartidos['mins4_A'] = minsA / (npj*11)

In [8]:
# matches_stats.to_csv(path_rawdata+'match_stats.csv',sep=';')
matches_stats = pd.read_csv(path_rawdata+'match_stats.csv',sep=';',index_col='matchId')
matches_stats[:3]

,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A,mins4_H,mins4_A
matchId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,53.071429,86.500000,0.285714,0.071429,48.571429,84.857143,0.428571,0.142857,65.000000,67.386364
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,46.071429,82.928571,0.214286,0.000000,45.133333,84.600000,0.400000,0.000000,60.181818,44.568182
2576329,2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,...,47.833333,83.000000,0.666667,0.166667,48.181818,86.818182,0.363636,0.000000,63.113636,55.204545


In [241]:
matches_stats.mins4_H[2565632]

0.0

In [242]:
matches_wUltPartidos.mins4_H[2565632]

63.54545454545455

In [243]:
min(matches_wUltPartidos.mins4_A)

22.318181818181817

### AÑADIR LOS VECTORES DE LAS DEMÁS CARACTERÍSTICAS

1) Sacamos de cada equipo cuantos disparos/pases/faltas ha hecho en los ultimos N partidos los jugadores del 11 titular, y luego lo dividimos entre el numero de minutos que hayan jugado los titulares.

In [244]:
passes

,id,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,y0,x0,y1,x1,accuracy,keyPass
0,180423957,8,Simple pass,[1801],8327,"[52, 49, 44, 44]",2575959,Pass,3158,1H,2.530536,85,52,49,44,44,1,0
1,180423958,8,Simple pass,[1801],20438,"[44, 43, 17, 17]",2575959,Pass,3158,1H,3.768418,85,44,43,17,17,1,0
2,180423962,8,Simple pass,[1801],8306,"[15, 72, 25, 25]",2575959,Pass,3158,1H,10.376066,85,15,72,25,25,1,0
3,180423979,8,Simple pass,[1801],86366,"[82, 38, 88, 88]",2575959,Pass,3172,1H,16.240563,85,82,38,88,88,1,0
4,180423968,8,Simple pass,[1801],8306,"[15, 61, 15, 15]",2575959,Pass,3158,1H,19.153235,85,15,61,15,15,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565351,253302662,8,Hand pass,[1801],3486,"[29, 8, 19, 19]",2565927,Pass,682,2H,2925.547631,81,29,8,19,19,1,0
1565352,253302665,8,Simple pass,[1801],3695,"[34, 36, 52, 52]",2565927,Pass,682,2H,2931.782904,85,34,36,52,52,1,0
1565353,253302671,8,Simple pass,[1801],20623,"[25, 66, 2, 2]",2565927,Pass,682,2H,2939.077491,85,25,66,2,2,1,0
1565354,253302674,8,Cross,"[401, 1802]",122832,"[21, 97, 26, 26]",2565927,Pass,682,2H,2942.098761,80,21,97,26,26,0,0


In [245]:
def filterDB(db,matches,players):
    mask_players = False
    for p in players:
        mask_players = mask_players | np.array(db.playerId == p)

    mask_matches = False
    for m in matches:
        mask_matches = mask_matches | np.array(db.matchId == m)

    mask = mask_matches & mask_players
    
    return db[mask]

In [246]:
matches_stats.head(2)

,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A,mins4_H,mins4_A
matchId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,53.071429,86.500000,0.285714,0.071429,48.571429,84.857143,0.428571,0.142857,65.000000,67.386364
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,46.071429,82.928571,0.214286,0.000000,45.133333,84.600000,0.400000,0.000000,60.181818,44.568182


In [15]:
partidos_h, partidos_a = ult_partidos(4)

In [248]:
# line-up
players_mins_DF.loc[2576335].loc[3162].players_id[-11:]

array([ 21384,  20550,    130, 346908, 376362, 166534,  20561,  40806,
       265865, 228928,   7965])

In [249]:
players_mins_DF

players_id  \
matchesId teamsId                                                      
2576335   3162     [4792, 20972, 256479, 241675, 260250, 69417, 2...   
          3161     [3543, 405608, 20572, 86785, 7982, 70161, 2982...   
2576336   3158     [289122, 20438, 20518, 137298, 350032, 397239,...   
          3315     [423151, 267185, 22699, 392384, 302798, 415348...   
2576329   3173     [402898, 434142, 21299, 20636, 40538, 283832, ...   
...                                                              ...   
2565549   687      [3663, 3667, 225996, 477352, 3685, 3576, 3681,...   
2565556   674      [326535, 215265, 3843, 364844, 293862, 250885,...   
          714      [3372, 4350, 4362, 519496, 4361, 4238, 86200, ...   
2565557   696      [279541, 286981, 224080, 91064, 114055, 3650, ...   
          712      [6513, 484931, 86716, 86350, 6365, 167118, 393...   

                                                         players_min  
matchesId teamsId                                                     
2576335   3162     [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 13, 15, 75, 77,...  
          3161     [0, 0, 0, 0, 0, 0, 0, 0, 23, 30, 11, 60, 67, 7...  
2576336   3158     [0, 0, 0, 0, 0, 0, 0, 0, 13, 16, 21, 69, 74, 7...  
          3315     [0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 8, 20, 59, 70,...  
2576329   3173     [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 19, 15, 71, 75,...  
...                                                              ...  
2565549   687      [0, 0, 0, 0, 33, 23, 8, 57, 67, 82, 90, 90, 90...  
2565556   674      [0, 0, 0, 0, 17, 11, 1, 73, 79, 89, 90, 90, 90...  
          714      [0, 0, 0, 0, 12, 5, 12, 78, 78, 85, 90, 90, 90...  
2565557   696      [0, 0, 0, 0, 28, 27, 15, 62, 63, 75, 90, 90, 9...  
          712      [0, 0, 0, 0, 10, 25, 20, 65, 70, 80, 90, 90, 9...  

[3652 rows x 2 columns]

In [250]:
aux_filtered = filterDB(passes,partidos_a[2576336],players_mins_DF.loc[2576336].loc[3158].players_id[-11:])
count = len(aux_filtered)
acc = np.mean(aux_filtered.accuracy)
kp = sum(aux_filtered.keyPass)
print(count,acc,kp)

1131 0.8373121131741822 4


In [251]:
matches_wUltPartidos.head(1)

,matchId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,y1_ps_H,x1_ps_H,acc_avg_ps_H,keypass_ps_H,y0_ps_A,x0_ps_A,y1_ps_A,x1_ps_A,acc_avg_ps_A,keypass_ps_A
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,46.804878,46.804878,0.847561,3,44.05298,46.741722,43.966887,43.966887,0.824503,4


In [252]:
# creamos feature de pases y disparos segun el lineup-11

shots_11H, shots_acc_11H, goals_H = [], [], []
shots_11A, shots_acc_11A, goals_A = [], [], []

passes_11H, passes_acc_11H, keyPasses_H = [], [], []
passes_11A, passes_acc_11A, keyPasses_A = [], [], []

for m in matches_wUltPartidos.reset_index().itertuples():
    #shots
    db_filtered_h = filterDB(shots,partidos_h[m.wyId],players_mins_DF.loc[(m.wyId,m.teamId_home)].players_id[-11:])
    shots_11H.append(len(db_filtered_h))
    shots_acc_11H.append(np.mean(db_filtered_h.accuracy))
    goals_H.append(sum(db_filtered_h.goal))
    db_filtered_a = filterDB(shots,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    shots_11A.append(len(db_filtered_a))
    shots_acc_11A.append(np.mean(db_filtered_a.accuracy))
    goals_A.append(sum(db_filtered_a.goal))

    #passes
    db_filtered_h = filterDB(passes,partidos_h[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_home].players_id[-11:])
    passes_11H.append(len(db_filtered_h))
    passes_acc_11H.append(np.mean(db_filtered_h.accuracy))
    keyPasses_H.append(sum(db_filtered_h.keyPass))
    db_filtered_a = filterDB(passes,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    passes_11A.append(len(db_filtered_a))
    passes_acc_11A.append(np.mean(db_filtered_a.accuracy))
    keyPasses_A.append(sum(db_filtered_a.keyPass))



In [253]:
assert len(shots_11H) == len(shots_11A) == len(shots_acc_11H)

In [254]:
matches_wUltPartidos['shots_11H'] = shots_11H
matches_wUltPartidos['shots_11A'] = shots_11A
matches_wUltPartidos['shots_acc_11H'] = shots_acc_11H
matches_wUltPartidos['shots_acc_11A'] = shots_acc_11A
matches_wUltPartidos['goals_H'] = goals_H
matches_wUltPartidos['goals_A'] = goals_A
matches_wUltPartidos['passes_11H'] = passes_11H
matches_wUltPartidos['passes_11A'] = passes_11A
matches_wUltPartidos['passes_acc_11H'] = passes_acc_11H
matches_wUltPartidos['passes_acc_11A'] = passes_acc_11A
matches_wUltPartidos['keyPasses_H'] = keyPasses_H
matches_wUltPartidos['keyPasses_A'] = keyPasses_A

In [255]:
matches_wUltPartidos.columns

Index(['matchId', 'dateutc', 'competitionId', 'seasonId', 'roundId', 'winner',
       'teamId_home', 'score_home', 'scoreHT_home', 'teamId_away',
       'score_away', 'scoreHT_away', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'B365H', 'B365D', 'B365A', 'y0_sh_H', 'x0_sh_H',
       'acc_avg_sh_H', 'goal_avg_sh_H', 'y0_sh_A', 'x0_sh_A', 'acc_avg_sh_A',
       'goal_avg_sh_A', 'mins4_H', 'mins4_A', 'y0_ps_H', 'x0_ps_H', 'y1_ps_H',
       'x1_ps_H', 'acc_avg_ps_H', 'keypass_ps_H', 'y0_ps_A', 'x0_ps_A',
       'y1_ps_A', 'x1_ps_A', 'acc_avg_ps_A', 'keypass_ps_A', 'shots_11H',
       'shots_11A', 'shots_acc_11H', 'shots_acc_11A', 'goals_H', 'goals_A',
       'passes_11H', 'passes_11A', 'passes_acc_11H', 'passes_acc_11A',
       'keyPasses_H', 'keyPasses_A'],
      dtype='object')

In [256]:
print(matches_wUltPartidos.loc[2576335].acc_avg_sh_A, matches_wUltPartidos.loc[2576335].shots_acc_11A)

0.4285714285714285 0.45454545454545453


In [5]:
# matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')
matches_wUltPartidos = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')
matches_wUltPartidos

,matchId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,passes_11H,passes_11A,passes_acc_11H,passes_acc_11A,keyPasses_H,keyPasses_A,ataque_defensa_h,ataque_defensa_a,ataque_defensa_season_h,ataque_defensa_season_a
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,1207,1697,0.816073,0.833235,8,12,0.873867,1.380648,0.865304,1.138606
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,871,1131,0.731343,0.837312,9,4,-0.098474,2.868924,-0.801495,1.196590
2576329,2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,...,1066,1350,0.742026,0.849630,9,11,-0.746701,2.868924,-0.792723,0.767209
2576330,2576330,2018-05-20 16:00:00,524,181248,4406278,3165,3165,1,0,3219,...,937,1197,0.786553,0.861320,3,2,-0.422588,-0.479696,-0.908570,-0.929413
2576331,2576331,2018-05-20 16:00:00,524,181248,4406278,3163,3163,1,1,3166,...,905,869,0.805525,0.806674,8,7,-0.811524,-0.702938,-0.576843,-0.579486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565632,2565632,2017-10-21 16:30:00,795,181144,4406122,674,674,4,1,680,...,1175,1028,0.864681,0.824903,7,6,1.279009,-0.355673,0.598524,-0.560691
2565634,2565634,2017-10-21 14:15:00,795,181144,4406122,684,684,2,1,696,...,1284,839,0.891745,0.800954,10,4,-0.240274,-0.355673,-0.230614,-0.148718
2565628,2565628,2017-10-21 11:00:00,795,181144,4406122,0,695,1,0,698,...,935,886,0.829947,0.698646,4,5,-0.325354,-0.107627,-0.589635,0.266265


In [258]:
matches_wUltPartidos.mins4_H

wyId
2576335    65.000000
2576336    60.181818
2576329    63.113636
2576330    53.863636
2576331    56.590909
             ...    
2565632    63.545455
2565634    54.590909
2565628    53.340909
2565618    65.727273
2565614    65.272727
Name: mins4_H, Length: 1434, dtype: float64

In [6]:
matches_wUltPartidos.columns

Index(['matchId', 'dateutc', 'competitionId', 'seasonId', 'roundId', 'winner',
       'teamId_home', 'score_home', 'scoreHT_home', 'teamId_away',
       'score_away', 'scoreHT_away', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'B365H', 'B365D', 'B365A', 'y0_sh_H', 'x0_sh_H',
       'acc_avg_sh_H', 'goal_avg_sh_H', 'y0_sh_A', 'x0_sh_A', 'acc_avg_sh_A',
       'goal_avg_sh_A', 'mins4_H', 'mins4_A', 'y0_ps_H', 'x0_ps_H', 'y1_ps_H',
       'x1_ps_H', 'acc_avg_ps_H', 'keypass_ps_H', 'y0_ps_A', 'x0_ps_A',
       'y1_ps_A', 'x1_ps_A', 'acc_avg_ps_A', 'keypass_ps_A', 'shots_11H',
       'shots_11A', 'shots_acc_11H', 'shots_acc_11A', 'goals_H', 'goals_A',
       'passes_11H', 'passes_11A', 'passes_acc_11H', 'passes_acc_11A',
       'keyPasses_H', 'keyPasses_A', 'ataque_defensa_h', 'ataque_defensa_a',
       'ataque_defensa_season_h', 'ataque_defensa_season_a'],
      dtype='object')

## INDICADOR ATAQUE-DEFENSA

avg. goles marcados / avg. goles encajados   [ ult. 4 partidos ]

In [260]:
teams.loc[1673]

city                                 Huddersfield, West Yorkshire
name                                            Huddersfield Town
officialName                                 Huddersfield Town FC
area            {'name': 'England', 'id': '0', 'alpha3code': '...
type                                                         club
Name: 1673, dtype: object

In [261]:
partidos_h

{2576335: array([2576309, 2576297, 2576274, 2576249], dtype=int64),
 2576336: array([2576317, 2576292, 2576269, 2576253], dtype=int64),
 2576329: array([2576316, 2576290, 2576278, 2576257], dtype=int64),
 2576330: array([2576312, 2576294, 2576277, 2576228], dtype=int64),
 2576331: array([2576315, 2576291, 2576267, 2576222], dtype=int64),
 2576332: array([2576318, 2576288, 2576262, 2576244], dtype=int64),
 2576333: array([2576314, 2576289, 2576273, 2576268], dtype=int64),
 2576337: array([2576310, 2576296, 2576282, 2576259], dtype=int64),
 2576338: array([2576313, 2576293, 2576270, 2576220], dtype=int64),
 2576334: array([2576311, 2576295, 2576275, 2576252], dtype=int64),
 2576322: array([2576300, 2576283, 2576263, 2576237], dtype=int64),
 2576325: array([2576299, 2576280, 2576264, 2576245], dtype=int64),
 2576324: array([2576301, 2576298, 2576272, 2576227], dtype=int64),
 2576319: array([2576305, 2576285, 2576276, 2576250], dtype=int64),
 2576320: array([2576304, 2576271, 2576254, 2576

In [262]:
matches_stats.head(2)

,wyId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,y0_sh_H,x0_sh_H,acc_avg_sh_H,goal_avg_sh_H,y0_sh_A,x0_sh_A,acc_avg_sh_A,goal_avg_sh_A,mins4_H,mins4_A
matchId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,53.071429,86.500000,0.285714,0.071429,48.571429,84.857143,0.428571,0.142857,65.000000,67.386364
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,46.071429,82.928571,0.214286,0.000000,45.133333,84.600000,0.400000,0.000000,60.181818,44.568182


In [16]:
ataque_defensa = [[],[]]

for match in matches_wUltPartidos.itertuples(): 
    df_ult_partH = matches_stats.loc[partidos_h[match.matchId]][['teamId_home','teamId_away','score_away','score_home']]
    df_ult_partA = matches_stats.loc[partidos_a[match.matchId]][['teamId_home','teamId_away','score_away','score_home']]

    df_ult = [df_ult_partH, df_ult_partA]
    col    = ['teamId_home','teamId_away']
    score  = ['score_away','score_home']

    for i,df in enumerate(df_ult):
        # avg_encajados, avg_marcados = np.mean(df[score[i]]), np.mean(df[score[not(i)]])
        avg_encajados, avg_marcados = np.sum(df[score[i]]), np.mean(df[score[not(i)]])
        if(avg_encajados > 0):
            ataque_defensa[i].append(avg_marcados/avg_encajados)   
        else: ataque_defensa[i].append(avg_marcados)

# un equipo que solo ha recibido un gol y ha marcado X goles recibirá mejor puntuación
# que otro equipo que ha marcado los mismos goles pero no ha encajado ni un gol.
# si dividimos por la SUMA de los goles encajados desaparece este problema.
# sin embargo, un equipo que recibe un solo gol y no marca ninguno, 
# recibe menos puntuacion que otro que marca un gol y recibe muchos.

In [17]:
assert len(ataque_defensa[0]) == len(matches_wUltPartidos)

In [18]:
ataque_defensa_h = np.array(ataque_defensa[0])
ataque_defensa_a = np.array(ataque_defensa[1])

matches_wUltPartidos['ataque_defensa_h'] = (ataque_defensa_h-np.mean(ataque_defensa_h))/np.std(ataque_defensa_h)
matches_wUltPartidos['ataque_defensa_a'] = (ataque_defensa_a-np.mean(ataque_defensa_a))/np.std(ataque_defensa_a)

In [22]:
max(matches_wUltPartidos.ataque_defensa_h)

6.533224454626111

In [30]:
ataque_defensa_h.max()

4.25

In [267]:
matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')

In [268]:
avg_encajados_home = matches_wUltPartidos.groupby('teamId_home').agg(encajados=('score_away','mean'))
avg_marcados_home  = matches_wUltPartidos.groupby('teamId_home').agg(marcados=('score_home','mean'))
avg_encajados_away = matches_wUltPartidos.groupby('teamId_away').agg(encajados=('score_home','mean'))
avg_marcados_away  = matches_wUltPartidos.groupby('teamId_away').agg(marcados=('score_away','mean'))

avg_encajados = (pd.concat([avg_encajados_away,avg_encajados_home],axis=1)).mean(axis=1)  # (encajados como locales + commo encajados visit) / 2 -> pq son mismo # partidos
avg_marcados  = (pd.concat([avg_marcados_home,avg_marcados_away],axis=1)).mean(axis=1)

ataque_defensa = (avg_marcados/avg_encajados).sort_index()
ataque_defensa_norm = (ataque_defensa-ataque_defensa.mean())/ataque_defensa.std()

ataque_defensa_teams = pd.DataFrame(ataque_defensa_norm,columns=['indicador']).reset_index()
teamId = ataque_defensa_teams['index']
ataque_defensa_teams['teamId'] = teamId
ataque_defensa_teams.drop('index',axis=1,inplace=True)


In [269]:
ataque_defensa_teams.set_index('teamId',inplace=True)

In [270]:
ataque_defensa_teams

,indicador
teamId,
674,0.598524
675,1.385170
676,2.431061
677,-0.919901
678,-0.512883
...,...
3795,-0.872641
3799,-0.573298
3804,-0.473012


In [271]:
ad_h, ad_a = [], []

for match in matches_wUltPartidos.itertuples():
    ind_h = ataque_defensa_teams.loc[match.teamId_home].indicador
    ind_a = ataque_defensa_teams.loc[match.teamId_away].indicador
    ad_h.append(ind_h), ad_a.append(ind_a)
    
matches_wUltPartidos['ataque_defensa_season_h'] = ad_h
matches_wUltPartidos['ataque_defensa_season_a'] = ad_a

In [272]:
matches_wUltPartidos

,matchId,dateutc,competitionId,seasonId,roundId,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,passes_11H,passes_11A,passes_acc_11H,passes_acc_11A,keyPasses_H,keyPasses_A,ataque_defensa_h,ataque_defensa_a,ataque_defensa_season_h,ataque_defensa_season_a
wyId,,,,,,,,,,,,,,,,,,,,,
2576335,2576335,2018-05-20 18:45:00,524,181248,4406278,3161,3162,2,2,3161,...,1207,1697,0.816073,0.833235,8,12,0.873867,1.380648,0.865304,1.138606
2576336,2576336,2018-05-20 18:45:00,524,181248,4406278,3158,3315,0,0,3158,...,871,1131,0.731343,0.837312,9,4,-0.098474,2.868924,-0.801495,1.196590
2576329,2576329,2018-05-20 16:00:00,524,181248,4406278,3173,3173,1,0,3172,...,1066,1350,0.742026,0.849630,9,11,-0.746701,2.868924,-0.792723,0.767209
2576330,2576330,2018-05-20 16:00:00,524,181248,4406278,3165,3165,1,0,3219,...,937,1197,0.786553,0.861320,3,2,-0.422588,-0.479696,-0.908570,-0.929413
2576331,2576331,2018-05-20 16:00:00,524,181248,4406278,3163,3163,1,1,3166,...,905,869,0.805525,0.806674,8,7,-0.811524,-0.702938,-0.576843,-0.579486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565632,2565632,2017-10-21 16:30:00,795,181144,4406122,674,674,4,1,680,...,1175,1028,0.864681,0.824903,7,6,1.279009,-0.355673,0.598524,-0.560691
2565634,2565634,2017-10-21 14:15:00,795,181144,4406122,684,684,2,1,696,...,1284,839,0.891745,0.800954,10,4,-0.240274,-0.355673,-0.230614,-0.148718
2565628,2565628,2017-10-21 11:00:00,795,181144,4406122,0,695,1,0,698,...,935,886,0.829947,0.698646,4,5,-0.325354,-0.107627,-0.589635,0.266265
